# Set up

Create a regression problem

In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

%matplotlib inline

In [2]:
X, y = make_regression(n_samples=1000)

r2_scores = dict()  # We will hold results here.

Use a Random Forest to model the problem

In [3]:
model = RandomForestRegressor(oob_score=True, n_estimators=100, n_jobs=-1)

In [14]:
model.fit(X, y)
r2_scores['Benchmark'] = model.oob_score_
pd.Series(r2_scores)

Benchmark           0.678078
Pipeline Overfit    0.979797
dtype: float64

# Use the Transformer to calibrate

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin

from QuantileCalibrator import QuantileCalibrator

# Hacky way to change a RandomForest into a Transformer
class RandomForestTransformer(RandomForestRegressor, TransformerMixin):
    
    def transform(self, X, y=None):
        return self.predict(X)

In [9]:
rf = RandomForestTransformer(oob_score=True, n_estimators=100)
qc = QuantileCalibrator(quantile=100, isotonic_fit=True, isotonic_lambda=1)

steps = [
    ('random_forest', rf),
    ('quantile_cal', qc)
]

pipeline = Pipeline(steps=steps)

In [10]:
pipeline.fit(X, y)

Pipeline(memory=None,
     steps=[('random_forest', RandomForestTransformer(bootstrap=True, criterion='mse', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction...t=False)), ('quantile_cal', QuantileCalibrator(isotonic_fit=True, isotonic_lambda=1, quantile=100))])

Scoring like this will result in over fitting:

In [13]:
r2_scores['Pipeline Overfit'] = pipeline.score(X, y)
pd.Series(r2_scores)

Benchmark           0.681189
Pipeline Overfit    0.979797
dtype: float64

We can instead use the out-of-bag predictions:

In [15]:
r2_scores['Pipeline OOB'] = qc.score(rf.oob_prediction_, y)
pd.Series(r2_scores)

Benchmark           0.678078
Pipeline OOB        0.738246
Pipeline Overfit    0.979797
dtype: float64

# Cross Validate Results

Alternatively, we can use $k$-fold cross validation on the entire pipeline

In [17]:
from sklearn.model_selection import cross_validate, cross_val_score

In [19]:
cross_validated_scores = cross_val_score(X=X, y=y, cv=10, estimator=pipeline, n_jobs=-1)
r2_scores['Pipeline 10 Fold CV'] = cross_validated_scores.mean()
pd.Series(r2_scores)

Benchmark              0.678078
Pipeline 10 Fold CV    0.718343
Pipeline OOB           0.738246
Pipeline Overfit       0.979797
dtype: float64

# Hyper Parameter Search: 2 Steps

We can now optimize to find the best hyper parameters.

First, we'll do this with only the Random Forest.

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [40]:
search_params = {
    'n_estimators': randint(10, 1000),
    'max_features': uniform(0, 1)
}

rf = RandomForestRegressor()

random_search = RandomizedSearchCV(estimator=rf, param_distributions=search_params, n_iter=30, n_jobs=-1, cv=10)

In [41]:
search_result = random_search.fit(X, y)
search_result.best_params_

{'max_features': 0.8041407631899328, 'n_estimators': 892}

In [42]:
# Train on the full dataset
rf.set_params(oob_score=True, **search_result.best_params_)
rf.fit(X, y)
r2_scores['RF Only, Best HPs'] = rf.oob_score_
pd.Series(r2_scores)

Benchmark                   0.678078
Pipeline 10 Fold CV         0.718343
Pipeline Best HP 2 steps    0.744184
Pipeline OOB                0.738246
Pipeline Overfit            0.979797
RF Only, Best HPs           0.694573
dtype: float64

Next, we can fit a quantile calibrator using these parameters!

In [43]:
rf = RandomForestTransformer()
qc = QuantileCalibrator()

pipeline = Pipeline(steps=[('random_forest', rf), ('quantile_calibrator', qc)], memory='.')

# We only need to fit params for the QuantileCalibrator because the RandomForest was already fit above.
search_params = {
    'random_forest__max_features': [search_result.best_params_['max_features']],
    'random_forest__n_estimators': [search_result.best_params_['n_estimators']],
    'quantile_calibrator__quantile': randint(10, 300),
    'quantile_calibrator__isotonic_fit': [True, False],
    'quantile_calibrator__isotonic_lambda': uniform(0.01, 20)
}

random_search = RandomizedSearchCV(estimator=pipeline, 
                                   param_distributions=search_params, 
                                   n_iter=30, 
                                   n_jobs=-1,
                                   verbose=1,
                                   cv=10)

In [44]:
search_result2 = random_search.fit(X, y)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


/opt/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:213: UserWarning: Persisting input arguments took 1.76s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/opt/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:213: UserWarning: Persisting input arguments took 2.64s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/opt/anaconda3/l

/opt/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:213: UserWarning: Persisting input arguments took 3.26s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/opt/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:213: UserWarning: Persisting input arguments took 3.26s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/opt/anaconda3/l

/opt/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:213: UserWarning: Persisting input arguments took 0.67s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/opt/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:213: UserWarning: Persisting input arguments took 1.41s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
[Parallel(n_jobs

In [45]:
search_result2.best_params_

{'quantile_calibrator__isotonic_fit': True,
 'quantile_calibrator__isotonic_lambda': 5.218566439308666,
 'quantile_calibrator__quantile': 45,
 'random_forest__max_features': 0.8041407631899328,
 'random_forest__n_estimators': 892}

In [46]:
# Train on the full dataset
pipeline.set_params(random_forest__oob_score=True, **search_result2.best_params_)
pipeline.fit(X, y)
rf_pred = pipeline.named_steps['random_forest'].oob_prediction_
r2_scores['Pipeline Best HP 2 steps'] = pipeline.named_steps['quantile_calibrator'].score(rf_pred, y)
pd.Series(r2_scores)

Benchmark                   0.678078
Pipeline 10 Fold CV         0.718343
Pipeline Best HP 2 steps    0.745952
Pipeline OOB                0.738246
Pipeline Overfit            0.979797
RF Only, Best HPs           0.694573
dtype: float64

# Hyper Parameter Search: 1 Step

We can also search for the best HPs for both stages of the pipeline simultaniously.

In [47]:
rf = RandomForestTransformer()
qc = QuantileCalibrator()

pipeline = Pipeline(steps=[('random_forest', rf), ('quantile_calibrator', qc)], memory='.')

# We only need to fit params for the QuantileCalibrator because the RandomForest was already fit above.
search_params = {
    'random_forest__max_features': uniform(0.1, 0.9),
    'random_forest__n_estimators': randint(10, 1000),
    'random_forest__n_jobs': [-1],
    'quantile_calibrator__quantile': randint(10, 300),
    'quantile_calibrator__isotonic_fit': [True, False],
    'quantile_calibrator__isotonic_lambda': uniform(0.01, 20)
}

random_search = RandomizedSearchCV(estimator=pipeline, 
                                   param_distributions=search_params, 
                                   n_iter=30, 
                                   n_jobs=-1,
                                   verbose=1,
                                   cv=10)

In [48]:
search_result3 = random_search.fit(X, y)
search_result3.best_params_

Fitting 10 folds for each of 30 candidates, totalling 300 fits


/opt/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:213: UserWarning: Persisting input arguments took 0.66s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/opt/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:213: UserWarning: Persisting input arguments took 0.84s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
[Parallel(n_jobs

{'quantile_calibrator__isotonic_fit': True,
 'quantile_calibrator__isotonic_lambda': 17.021233519809098,
 'quantile_calibrator__quantile': 188,
 'random_forest__max_features': 0.4543227398403513,
 'random_forest__n_estimators': 851,
 'random_forest__n_jobs': -1}

In [50]:
r2_scores['Pipeline Best HP 1 step'] = search_result3.best_score_.mean()
pd.Series(r2_scores)

Benchmark                   0.678078
Pipeline 10 Fold CV         0.718343
Pipeline Best HP 1 step     0.737662
Pipeline Best HP 2 steps    0.745952
Pipeline OOB                0.738246
Pipeline Overfit            0.979797
RF Only, Best HPs           0.694573
dtype: float64